# **Data initialisation**

In [ ]:
# Installer les dépendances
!pip install python-dotenv
!pip install supabase
!pip install pandas-profiling
!pip install category_encoders
!pip install mlflow
!pip install pyngrok
!pip install PyGithub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.4/216.4 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.8 MB/s eta 0:00:0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 8.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 102.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.5/83.5 kB 13.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.3/224.3 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.1/148.1 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 11.0 MB/s eta 0:00:00
  Created wheel for databricks-cli: filename=databricks_cli-0.17.7-py3-none-any.whl size=143860 sha256=3c8af99902ae4518351fdcfb27e0f697bd651ba2e05ef6c3819a2b1769cd73a0
  Stored in directory: /roo

In [ ]:
# Importer les bibliothèques nécessaires
import requests
import csv
import os
import mlflow
import time
import pickle
import glob
import warnings
import tarfile
import fnmatch
import numpy as np
import pandas as pd
import category_encoders as ce
from github import Github
from google.colab import drive
from supabase import create_client, Client
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from pandas_profiling import ProfileReport
from pyngrok import ngrok

<ipython-input-2-6e7df58160e4>:21: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  from pandas_profiling import ProfileReport


In [ ]:
# Configurer l'environnement
warnings.filterwarnings('ignore')
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Projet MLOps/')
load_dotenv('config.env')

Mounted at /content/drive


True

# **Data extraction**

In [ ]:
# data_extraction.py
def download_csv_files(url, file_patterns, files_filepath):
    print("Récupération du contenu HTML de l'URL...")
    html_content = requests.get(url).text

    print("Analyse du contenu HTML avec BeautifulSoup...")
    soup = BeautifulSoup(html_content, 'html.parser')

    print("Récupération des liens vers les fichiers CSV dans le contenu HTML...")
    csv_links = [a['href'] for a in soup.find_all('a', href=True) if a['href'].endswith('.csv')]
    
    print("Tentative de récupération de la liste des fichiers déjà téléchargés...")
    try:
        with open(files_filepath, "r") as f:
            downloaded_files = f.read().splitlines()
    except FileNotFoundError:
        print("Le fichier de la liste des téléchargements n'a pas été trouvé. Création d'une nouvelle liste...")
        downloaded_files = []
    
    print("Initialisation de la liste des nouveaux fichiers téléchargés...")
    new_files = []
    
    print("Parcours des liens vers les fichiers CSV...")
    for link in csv_links:
        filename = os.path.basename(link)
        if any(pattern in filename for pattern in file_patterns) and filename not in downloaded_files:
            print(f"Téléchargement et sauvegarde du fichier {filename}...")
            content = requests.get(link).content
            with open(filename, "wb") as f:
                f.write(content)
            new_files.append(filename)
            print(f"Fichier {filename} téléchargé avec succès !")
        else:
            print(f"Le fichier {filename} a déjà été téléchargé ou ne correspond pas aux motifs de fichier. Il ne sera pas téléchargé.")

    print("Ajout des nouveaux fichiers à la liste des fichiers déjà téléchargés...")
    downloaded_files += new_files

    print("Sauvegarde de la liste mise à jour des fichiers téléchargés...")
    with open(files_filepath, "w") as f:
        f.write("\n".join(downloaded_files))


def export_to_csv(supabase, table_name, output_file):
    print(f"Récupération des données de la table '{table_name}'...")
    response = supabase.table(table_name).select("*").execute()
    data = response.data

    if not data:
        print(f"Aucune donnée trouvée dans la table '{table_name}'.")
        return None, None
    
    print("Récupération des noms de colonnes...")
    columns = data[0].keys()

    print(f"Écriture des données dans le fichier CSV '{output_file}'...")
    with open(output_file, mode='w', encoding='utf-8', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=columns)
        writer.writeheader()
        for row in data:
            writer.writerow(row)

In [ ]:
## Tâche pour extraire les données "live" à partir de Supabase
def data_extraction_live_func():
    print("Initialisation du téléchargement des données live...")
    url: str = os.getenv("SUPABASE_URL")
    key: str = os.getenv("SUPABASE_KEY")
    supabase: Client = create_client(url, key)
    table_name = "data_live"
    output_file = "data_live.csv"
    export_to_csv(supabase, table_name, output_file)
    print("Téléchargement des données live terminé.")

## Tâche pour extraire les nouvelles données historiques à partir du site web
def data_extraction_historical_new_func():
    print("Initialisation du téléchargement des nouvelles données historiques...")
    url = "https://www.data.gouv.fr/en/datasets/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2021/"
    file_patterns = ['usagers-', 'lieux-', 'cteristiques-', 'vehicules-']
    files_filepath = "downloaded_files.txt"
    download_csv_files(url, file_patterns, files_filepath)
    print("Téléchargement des nouvelles données historiques terminé.")

## Tâche pour extraire les anciennes données historiques à partir de Supabase
def data_extraction_historical_old_func():
    print("Initialisation du téléchargement des anciennes données historiques...")
    url: str = os.getenv("SUPABASE_URL")
    key: str = os.getenv("SUPABASE_KEY")
    supabase: Client = create_client(url, key)
    table_name = "data_historical"
    output_file = "data_historical_old.csv"
    export_to_csv(supabase, table_name, output_file)
    print("Téléchargement des anciennes données historiques terminé.")

In [ ]:
data_extraction_live_func()
data_extraction_historical_new_func()
data_extraction_historical_old_func()

Initialisation du téléchargement des données live...
Récupération des données de la table 'data_live'...
Aucune donnée trouvée dans la table 'data_live'.
Téléchargement des données live terminé.
Initialisation du téléchargement des nouvelles données historiques...
Récupération du contenu HTML de l'URL...
Analyse du contenu HTML avec BeautifulSoup...
Récupération des liens vers les fichiers CSV dans le contenu HTML...
Tentative de récupération de la liste des fichiers déjà téléchargés...
Le fichier de la liste des téléchargements n'a pas été trouvé. Création d'une nouvelle liste...
Initialisation de la liste des nouveaux fichiers téléchargés...
Parcours des liens vers les fichiers CSV...
Téléchargement et sauvegarde du fichier usagers-2021.csv...
Fichier usagers-2021.csv téléchargé avec succès !
Téléchargement et sauvegarde du fichier vehicules-2021.csv...
Fichier vehicules-2021.csv téléchargé avec succès !
Téléchargement et sauvegarde du fichier lieux-2021.csv...
Fichier lieux-2021.csv

# **Data merging**

In [ ]:
# data_merging.py
def upload_to_supabase(csv_file_path, chunk_size, supabase, table_name):
    print("Lecture du fichier CSV et préparation pour l'envoi à Supabase...")
    data_iterator = pd.read_csv(csv_file_path, chunksize=chunk_size, na_filter=False)
    row_count = 0
    for chunk in data_iterator:
        chunk_data = chunk.to_dict(orient='records')
        response = supabase.table(table_name).insert(chunk_data).execute()
        if response:
            print(f"{len(chunk_data)} lignes insérées avec succès dans la table '{table_name}'")
            row_count += len(chunk_data)
    print(f"{row_count} lignes insérées au total dans la table '{table_name}'")


def csv_to_dataframe(pattern: str, sample_size: int = 1024, encoding: str = 'latin1', low_memory: bool = False):
    print("Recherche du fichier CSV le plus récent correspondant au motif donné...")
    matching_files = glob.glob(pattern)
    if not matching_files:
        print("Aucun fichier correspondant trouvé.")
        return None
    most_recent_file = max(matching_files, key=os.path.getmtime)
    
    print(f"Le fichier CSV le plus récent trouvé est : {most_recent_file}")
    print("Détection du délimiteur du fichier CSV...")
    try:
        with open(most_recent_file, 'r', encoding=encoding) as csvfile:
            sample = csvfile.read(sample_size)
            sniffer = csv.Sniffer()
            delimiter = sniffer.sniff(sample).delimiter
    except Exception as e:
        print(f"Une erreur est survenue lors de la détection du délimiteur : {e}")
        return None

    print(f"Le délimiteur du fichier CSV est : {delimiter}")
    print("Lecture du fichier CSV...")
    try:
        df = pd.read_csv(most_recent_file, sep=delimiter, encoding=encoding, low_memory=low_memory)
    except Exception as e:
        print(f"Une erreur est survenue lors de la lecture du fichier CSV : {e}")
        return None

    print("Le fichier CSV a été lu avec succès.")
    return df


def merge_dataframes(df_list, merge_keys, merge_how='left'):
    print("Fusion des dataframes...")
    if not df_list:
        print("La liste des dataframes est vide.")
        return None
    if len(df_list) == 1:
        print("Une seule dataframe a été fournie. Aucune fusion nécessaire.")
        return df_list[0]

    merged_data = df_list[0]
    for i in range(1, len(df_list)):
        print(f"Fusion de la dataframe {i+1}...")
        merged_data = pd.merge(merged_data, df_list[i], how=merge_how, on=merge_keys[i-1])

    print("Les dataframes ont été fusionnées avec succès.")
    return merged_data

In [ ]:
## Tâche pour fusionner les nouvelles données historiques, les anciènnes données historiques et les données "live"
def data_merging_func():
    print("Initialisation de la fusion des données...")
    data_usagers = csv_to_dataframe(pattern="*usagers*.csv")
    data_vehicules = csv_to_dataframe(pattern="*vehicules*.csv")
    data_caracteristiques = csv_to_dataframe(pattern="*cteristiques*.csv")
    data_lieux = csv_to_dataframe(pattern="*lieux*.csv")
    if data_usagers is not None and data_vehicules is not None and data_caracteristiques is not None and data_lieux is not None:
        df_list = [data_usagers, data_vehicules, data_caracteristiques, data_lieux]
        merge_keys = [["Num_Acc", "num_veh"], ["Num_Acc"], ["Num_Acc"]]
        data_historical_new = merge_dataframes(df_list, merge_keys)
        data_historical_new.to_csv("data_historical_new.csv", index=False, encoding='utf-8')

        url: str = os.getenv("SUPABASE_URL")
        key: str = os.getenv("SUPABASE_KEY")
        supabase: Client = create_client(url, key)
        upload_to_supabase(
            csv_file_path = "data_historical_new.csv",
            chunk_size = 1000,
            supabase = supabase,
            table_name = "data_historical",
        )

        if os.path.isfile("data_historical_old.csv"):
            data_historical_old = pd.read_csv("data_historical_old.csv", sep=',', encoding='latin1', low_memory=False)
            data_historical = pd.concat([data_historical_new, data_historical_old], axis=0, ignore_index=True)
        else:
            data_historical = data_historical_new
    else:
        data_historical = pd.read_csv("data_historical_old.csv", sep=',', encoding='latin1', low_memory=False)

    if os.path.isfile("data_live.csv"):
        data_live = pd.read_csv("data_live.csv", sep=',', encoding='latin1', low_memory=False)
        data_accidents = pd.concat([data_historical, data_live], axis=0, ignore_index=True)
    else:
        data_accidents = data_historical

    data_accidents.to_csv("data_accidents.csv", index=False, encoding='utf-8')
    print("Fusion des données terminée.")

In [ ]:
data_merging_func()

Initialisation de la fusion des données...
Recherche du fichier CSV le plus récent correspondant au motif donné...
Le fichier CSV le plus récent trouvé est : usagers-2021.csv
Détection du délimiteur du fichier CSV...
Le délimiteur du fichier CSV est : ;
Lecture du fichier CSV...
Le fichier CSV a été lu avec succès.
Recherche du fichier CSV le plus récent correspondant au motif donné...
Le fichier CSV le plus récent trouvé est : vehicules-2021.csv
Détection du délimiteur du fichier CSV...
Le délimiteur du fichier CSV est : ;
Lecture du fichier CSV...
Le fichier CSV a été lu avec succès.
Recherche du fichier CSV le plus récent correspondant au motif donné...
Le fichier CSV le plus récent trouvé est : carcteristiques-2021.csv
Détection du délimiteur du fichier CSV...
Le délimiteur du fichier CSV est : ;
Lecture du fichier CSV...
Le fichier CSV a été lu avec succès.
Recherche du fichier CSV le plus récent correspondant au motif donné...
Le fichier CSV le plus récent trouvé est : lieux-2021

# **Data profiling**

In [ ]:
# data_profiling.py
def generate_analysis_report(df: pd.DataFrame, output_file: str, title: str):
    print("Initialisation du rapport...")
    profile = ProfileReport(
        df,
        title=title,
        minimal=True,
        correlations=None,
        html={'style': {'full_width': True}},
    )
    print("Génération du rapport...")
    profile.to_file(output_file=output_file)

In [ ]:
## Tâche pour générer un rapport d'analyse exploratoire des données
def data_profiling_func():
    print("Initialisation de la création du rapport d'analyse exploratoire des données...")
    df = pd.read_csv("data_accidents.csv", sep=',', encoding='latin1', low_memory=False)
    output_file = "data_profile.html"
    title = "Analyse exploratoire des accidents routiers en France"
    generate_analysis_report(df, output_file, title)
    print("Création du rapport d'analyse exploratoire des données terminée.")

In [ ]:
data_profiling_func()

Initialisation de la création du rapport d'analyse exploratoire des données...
Initialisation du rapport...
Génération du rapport...


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Création du rapport d'analyse exploratoire des données terminée.


# **Data processing (all features)**

In [ ]:
# data_processing.py
def preprocess_data(df: pd.DataFrame, target):
    print("Prétraitement des données...")
    df = df[df[target].isin([1, 2, 3, 4])]
    df = df.groupby(target).apply(lambda x: x.fillna(x.mode().iloc[0])).reset_index(drop=True)
    df[target] = df[target].apply(lambda x: 0 if x in [1, 4] else 1)
    colonnes_a_supprimer = [colonne for colonne in df.columns if 'id_vehicule' in colonne]
    if colonnes_a_supprimer:
      df = df.drop(colonnes_a_supprimer, axis=1).drop_duplicates()
    return df


def encode_data(data, target):
    print("Encodage des données...")
    categorical_columns = data.select_dtypes(include=['object'])
    numerical_columns = data.select_dtypes(exclude=['object'])
    encoder = ce.TargetEncoder()
    encoded_data = encoder.fit_transform(categorical_columns, data[target])
    print("Encodage des données terminé.")

    print("Enregistrement de l'encodeur...")
    filename = f"targetencoder.pkl"
    with open(filename, 'wb') as file:
        pickle.dump(encoder, file)
    print("L'encodeur a été enregistré avec succès.")

    return pd.concat([numerical_columns, encoded_data], axis=1)

In [ ]:
## Tâche pour prétraiter et encoder les données
def data_processing1_func():
    data_file = "data_accidents.csv"
    df = pd.read_csv(data_file, sep=',', encoding='latin1', low_memory=False)

    target = "grav"
    df_preprocessed = preprocess_data(df, target)

    df_encoded = encode_data(df_preprocessed, target)

    output_file = "data_accidents_encoded.csv"
    df_encoded.to_csv(output_file, index=False, encoding='utf-8')

In [ ]:
data_processing1_func()

Prétraitement des données...
Encodage des données...
Encodage des données terminé.
Enregistrement de l'encodeur...
L'encodeur a été enregistré avec succès.


# **Data featuring**

In [ ]:
def train_model(experiment_name, df_encoded, target):
    current_experiment = mlflow.get_experiment_by_name(experiment_name)
    if current_experiment is None:
        experiment_id = mlflow.create_experiment(experiment_name)
        print("L'expérience a été créée.")
    else:
        experiment_id = current_experiment.experiment_id
        print("L'expérience existe déjà.")

    with mlflow.start_run(experiment_id = experiment_id):
        X = df_encoded.drop(target, axis=1)
        y = df_encoded[target]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        print("Les données ont été préparées.")

        model = XGBClassifier()
        model.fit(X_train, y_train)
        print("Le modèle a été entraîné avec les features sélectionnées.")

        importances = model.feature_importances_
        feature_importances = sorted(zip(X.columns, importances), key=lambda x: x[1], reverse=True)
        selected_features = [feature for feature, importance in feature_importances[:15]]
        print(f"Les 15 features les plus importantes sont : {selected_features}")

        mlflow.log_param("model", "XGBoost Classifier")
        mlflow.log_param("top15_feature", selected_features)
        print("Résultats de l'entraînement sauvegardés")
        
    print("Entraînement terminé")

In [ ]:
## Tâche pour entraîner et évaluer le modèle de prédiction
def data_featuring_func():
    data_file = "data_accidents_encoded.csv"
    df_encoded = pd.read_csv(data_file, sep=',', encoding='latin1', low_memory=False)

    experiment_name = "Accidents_Model"
    target = "grav"
    train_model(experiment_name, df_encoded, target)

In [ ]:
data_featuring_func()

L'expérience a été créée.
Les données ont été préparées.
Le modèle a été entraîné avec les features sélectionnées.
Les 15 features les plus importantes sont : ['long', 'lat', 'secu1', 'etatp', 'agg', 'place', 'obsm', 'sexe', 'catv', 'catu', 'voie', 'obs', 'col', 'catr', 'motor']
Résultats de l'entraînement sauvegardés
Entraînement terminé


# **Data processing (top 15 features)**

In [ ]:
# data_processing.py
def preprocess_data(df: pd.DataFrame, target, selected_features):
    print("Prétraitement des données...")
    df = df[df[target].isin([1, 2, 3, 4])]
    df = df.groupby(target).apply(lambda x: x.fillna(x.mode().iloc[0])).reset_index(drop=True)
    df[target] = df[target].apply(lambda x: 0 if x in [1, 4] else 1)
    df = df[selected_features].drop_duplicates()
    print("Prétraitement des données terminé.")
    return df


def encode_data(data, target):
    print("Encodage des données...")
    categorical_columns = data.select_dtypes(include=['object'])
    numerical_columns = data.select_dtypes(exclude=['object'])
    encoder = ce.TargetEncoder()
    encoded_data = encoder.fit_transform(categorical_columns, data[target])
    print("Encodage des données terminé.")

    print("Enregistrement de l'encodeur...")
    filename = f"targetencoder.pkl"
    with open(filename, 'wb') as file:
        pickle.dump(encoder, file)
    print("L'encodeur a été enregistré avec succès.")

    return pd.concat([numerical_columns, encoded_data], axis=1)

In [ ]:
## Tâche pour prétraiter et encoder les données
def data_processing2_func():
    print("Initialisation du traitement des données...")
    df = pd.read_csv("data_accidents.csv", sep=',', encoding='latin1', low_memory=False)
    target = "grav"
    selected_features = ["long", "lat", "secu1", "locp", "actp", "agg", "obsm", "etatp", "catv", "col", "place", "obs", "vma", "catu", "manv", "grav"]
    df_preprocessed = preprocess_data(df, target, selected_features)
    df_encoded = encode_data(df_preprocessed, target)
    df_encoded.to_csv("data_accidents_encoded.csv", index=False, encoding='utf-8')
    print("Traitement des données terminée.")

In [ ]:
data_processing2_func()

Initialisation du traitement des données...
Prétraitement des données...
Prétraitement des données terminé.
Encodage des données...
Encodage des données terminé.
Enregistrement de l'encodeur...
L'encodeur a été enregistré avec succès.
Traitement des données terminée.


# **Data modeling**

In [ ]:
# data_modeling.py
def train_model(experiment_name, df_encoded, target):
    print("Début de l'entraînement du modèle...")
    current_experiment = mlflow.get_experiment_by_name(experiment_name)
    if current_experiment is None:
        experiment_id = mlflow.create_experiment(experiment_name)
        print(f"Création d'une nouvelle expérience MLflow avec l'ID : {experiment_id}")
    else:
        experiment_id = current_experiment.experiment_id
        print(f"Utilisation de l'expérience MLflow existante avec l'ID : {experiment_id}")

    with mlflow.start_run(experiment_id = experiment_id):
        print("Préparation des données pour l'entraînement...")
        X = df_encoded.drop(target, axis=1)
        y = df_encoded[target]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        print("Construction et entraînement du modèle XGBClassifier...")
        model = XGBClassifier()
        model.fit(X_train, y_train)

        print("Prédiction et calcul des métriques...")
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average=None)
        recall = recall_score(y_test, y_pred, average=None)
        f1_scores = f1_score(y_test, y_pred, average=None)
        print(classification_report(y_test, y_pred))

        print("Enregistrement des paramètres et des métriques dans MLflow...")
        mlflow.log_param("model", "XGBoost Classifier")
        mlflow.log_metric("accuracy", accuracy)
        for label, p in enumerate(precision):
            mlflow.log_metric(f"precision_class_{label}", p)
        for label, r in enumerate(recall):
            mlflow.log_metric(f"recall_class_{label}", r)
        for label, f1 in enumerate(f1_scores):
            mlflow.log_metric(f"f1_score_class_{label}", f1)

        print("Sauvegarde du modèle...")
        filename = "xgboost.pkl"
        with open(filename, 'wb') as file:
            pickle.dump(model, file)

        print("Enregistrement des artefacts dans MLflow...")
        pkl_files = [f for f in os.listdir() if f.endswith(".pkl")]
        for pkl_file in pkl_files:
            print(f"Sauvegarde de l'artefact: {pkl_file}")
            mlflow.log_artifact(pkl_file)

In [ ]:
## Tâche pour entraîner et évaluer le modèle de prédiction
def data_modeling_func():
    print("Initialisation de la création du model de machine learning...")
    experiment_name = "Accidents_Model"
    df_encoded = pd.read_csv("data_accidents_encoded.csv", sep=',', encoding='latin1', low_memory=False)
    target = "grav"
    train_model(experiment_name, df_encoded, target)
    print("Création du model de machine learning terminée.")

In [ ]:
data_modeling_func()

Initialisation de la création du model de machine learning...
Début de l'entraînement du modèle...
Création d'une nouvelle expérience MLflow avec l'ID : 165545817927485843
Préparation des données pour l'entraînement...
Construction et entraînement du modèle XGBClassifier...
Prédiction et calcul des métriques...
              precision    recall  f1-score   support

           0       0.96      0.98      0.97     20254
           1       0.89      0.81      0.84      4509

    accuracy                           0.95     24763
   macro avg       0.92      0.89      0.91     24763
weighted avg       0.94      0.95      0.94     24763

Enregistrement des paramètres et des métriques dans MLflow...
Sauvegarde du modèle...
Enregistrement des artefacts dans MLflow...
Sauvegarde de l'artefact: targetencoder.pkl
Sauvegarde de l'artefact: xgboost.pkl
Création du model de machine learning terminée.


# **Data validation (modeling)**

In [ ]:
# data_validation.py
def check_metrics(experiment_name,): #ti):
    print("Récupération de l'expérience...")
    experiment = mlflow.get_experiment_by_name(experiment_name)

    print("Récupération du dernier run...")
    runs = mlflow.search_runs(
        experiment_ids=[experiment.experiment_id],
        max_results=1,
        order_by=["start_time desc"],
    )
    latest_run = runs.iloc[0]

    thresholds = {
        "accuracy": 0.9,
        "precision_class_0": 0.8,
        "precision_class_1": 0.8,
        "recall_class_0": 0.8,
        "recall_class_1": 0.8,
        "f1_score_class_0": 0.8,
        "f1_score_class_1": 0.8,
    }

    alert = False
    metrics = {}
    print("Vérification des métriques...")
    for metric_name, threshold in thresholds.items():
        metric_value = latest_run[f"metrics.{metric_name}"]
        metrics[metric_name] = metric_value
        if metric_value < threshold:
            print(f"Alerte déclenchée pour {metric_name}")
            alert = True

    if alert:
        print("Préparation du contenu de l'e-mail d'alerte...")
        email_content = "Les métriques suivantes sont en dessous des seuils définis:\n\n"
        for metric_name, metric_value in metrics.items():
            email_content += f"{metric_name}: {metric_value}\n"
        #ti.xcom_push(key="alert_email_content", value=email_content)
        return True
    else:
        print("Aucune alerte déclenchée.")
        return False

In [ ]:
def data_validation_func1():#ti):
    print("Initialisation de la vérification des métriques de performance...")
    experiment_name = "Accidents_Model"
    alert = check_metrics(experiment_name)
    print("Vérification des métriques de performance terminée...")
    if alert:
        return 'data_tuning_task'
    else:
        return 'data_deployment_task'

In [ ]:
data_validation_func1()

Initialisation de la vérification des métriques de performance...
Récupération de l'expérience...
Récupération du dernier run...
Vérification des métriques...
Aucune alerte déclenchée.
Vérification des métriques de performance terminée...


'data_deployment_task'

# **Data tuning**

In [ ]:
# data_tuning.py
def xgboost_fine_tuning(experiment_name, df_encoded, target, n_iter=100, cv=3):
    print("Commencement de l'exécution de la fonction xgboost_fine_tuning...")
    current_experiment = mlflow.get_experiment_by_name(experiment_name)
    if current_experiment is None:
        experiment_id = mlflow.create_experiment(experiment_name)
    else:
        experiment_id = current_experiment.experiment_id

    with mlflow.start_run(experiment_id = experiment_id):
        print("Début d'une nouvelle exécution MLflow...")
        X = df_encoded.drop(target, axis=1)
        y = df_encoded[target]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        print("Début de la recherche d'hyperparamètres avec RandomizedSearchCV...")
        model = XGBClassifier(objective='binary:logistic', random_state=42)
        param_grid = {
            "learning_rate": np.logspace(-3, 0, num=10),
            "max_depth": np.arange(1, 5),
            "n_estimators": np.arange(50, 150, step=10),
            "gamma": np.logspace(-8, 0, num=10),
            "min_child_weight": np.arange(1, 5),
            "subsample": np.linspace(0.5, 1, num=10),
            "colsample_bytree": np.linspace(0.5, 1, num=10),
            "reg_alpha": np.logspace(-8, 0, num=10),
            "reg_lambda": np.logspace(-8, 0, num=10),
        }
        random_search = RandomizedSearchCV(
            model, 
            param_distributions=param_grid, 
            n_iter=10,
            scoring="neg_mean_squared_error", 
            cv=3,
            n_jobs=-1, 
            random_state=42
        )
        random_search.fit(X_train, y_train)

        print("Terminé avec RandomizedSearchCV. Début du calcul des métriques et de la journalisation...")
        best_hyperparameters = random_search.best_params_
        best_model = random_search.best_estimator_
        y_pred = best_model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average=None)
        recall = recall_score(y_test, y_pred, average=None)
        f1_scores = f1_score(y_test, y_pred, average=None)
        print(classification_report(y_test, y_pred))

        mlflow.log_param("model", "XGBoost Classifier")
        mlflow.log_param("optimization_method", "RandomizedSearchCV")
        mlflow.log_params(best_hyperparameters)
        mlflow.log_metric("accuracy", accuracy)
        for label, p in enumerate(precision):
            mlflow.log_metric(f"precision_class_{label}", p)
        for label, r in enumerate(recall):
            mlflow.log_metric(f"recall_class_{label}", r)
        for label, f1 in enumerate(f1_scores):
            mlflow.log_metric(f"f1_score_class_{label}", f1)

        print("Sauvegarde du meilleur modèle...")
        filename = "xgboost.pkl"
        with open(filename, 'wb') as file:
            pickle.dump(best_model, file)

        print("Enregistrement des artefacts dans MLflow...")
        pkl_files = [f for f in os.listdir() if f.endswith(".pkl")]
        for pkl_file in pkl_files:
            print(f"Sauvegarde de l'artefact: {pkl_file}")
            mlflow.log_artifact(pkl_file)


In [ ]:
## Tâche pour améliorer les performances du modèle
def data_tuning_func():
    print("Initialisation du tuning des performances du model de machine learning...")
    experiment_name = "Accidents_Model"
    df_encoded = pd.read_csv("data_accidents_encoded.csv", sep=',', encoding='latin1', low_memory=False)
    target = "grav"
    xgboost_fine_tuning(experiment_name, df_encoded, target)
    print("Tuning des performances du model de machine learning terminée.")

In [ ]:
data_tuning_func()

Initialisation du tuning des performances du model de machine learning...
Commencement de l'exécution de la fonction xgboost_fine_tuning...
Début d'une nouvelle exécution MLflow...
Début de la recherche d'hyperparamètres avec RandomizedSearchCV...
Terminé avec RandomizedSearchCV. Début du calcul des métriques et de la journalisation...
Sauvegarde du meilleur modèle...
Enregistrement des artefacts dans MLflow...
Sauvegarde de l'artefact: targetencoder.pkl
Sauvegarde de l'artefact: xgboost.pkl
Tuning des performances du model de machine learning terminée.


# **Data validation (tuning)**

In [ ]:
# data_validation.py
def check_metrics(experiment_name,): #ti):
    print("Récupération de l'expérience...")
    experiment = mlflow.get_experiment_by_name(experiment_name)

    print("Récupération du dernier run...")
    runs = mlflow.search_runs(
        experiment_ids=[experiment.experiment_id],
        max_results=1,
        order_by=["start_time desc"],
    )
    latest_run = runs.iloc[0]

    thresholds = {
        "accuracy": 0.9,
        "precision_class_0": 0.8,
        "precision_class_1": 0.8,
        "recall_class_0": 0.8,
        "recall_class_1": 0.8,
        "f1_score_class_0": 0.8,
        "f1_score_class_1": 0.8,
    }

    alert = False
    metrics = {}
    print("Vérification des métriques...")
    for metric_name, threshold in thresholds.items():
        metric_value = latest_run[f"metrics.{metric_name}"]
        metrics[metric_name] = metric_value
        if metric_value < threshold:
            print(f"Alerte déclenchée pour {metric_name}")
            alert = True

    if alert:
        print("Préparation du contenu de l'e-mail d'alerte...")
        email_content = "Les métriques suivantes sont en dessous des seuils définis:\n\n"
        for metric_name, metric_value in metrics.items():
            email_content += f"{metric_name}: {metric_value}\n"
        #ti.xcom_push(key="alert_email_content", value=email_content)
        return True
    else:
        print("Aucune alerte déclenchée.")
        return False

In [ ]:
def data_validation_func2(): #ti):
    print("Initialisation de la vérification des métriques de performance...")
    experiment_name = "Accidents_Model"
    alert = check_metrics(experiment_name,) #ti)
    print("Vérification des métriques de performance terminée...")
    if alert:
        return 'data_alert_task'
    else:
        return 'data_deployment_task'

In [ ]:
data_validation_func2()

Initialisation de la vérification des métriques de performance...
Récupération de l'expérience...
Récupération du dernier run...
Vérification des métriques...
Aucune alerte déclenchée.
Vérification des métriques de performance terminée...


'data_deployment_task'

# **Data déployment**

In [ ]:
# data_deployment.py
def upload_to_github(input_path, token, repo_name, commit_message, branch, timestamp):
    print("Préparation pour l'envoi à Github...")
    g = Github(token)
    repo = g.get_user().get_repo(repo_name)

    local_file_path = input_path
    github_file_path = f"src/app/api/{timestamp}_{os.path.basename(local_file_path)}"
    with open(local_file_path, "rb") as file:
        content = file.read()
    try:
        repo.create_file(
            path=github_file_path,
            message=commit_message,
            content=content,
            branch=branch
        )
    except Exception as e:
        print(f"Erreur lors de la création du fichier '{github_file_path}' sur Github : {str(e)}")
    print(f"Le fichier '{github_file_path}' a été téléchargé avec succès sur le dépôt Github '{repo_name}'")

In [ ]:
## Tâche pour déployer le modèle de machine learning en production
def data_deployment_func():
    print("Initialisation du déploiement du model de machine learning en production...")
    timestamp = time.strftime('%Y%m%d_%H%M%S')
    token = os.getenv("GITHUB_TOKEN")

    targetencoder_files = glob.glob('./mlruns/**/targetencoder.pkl', recursive=True)
    most_recent_targetencoder_file = max(targetencoder_files, key=os.path.getmtime)
    upload_to_github(
        timestamp = timestamp,
        input_path = most_recent_targetencoder_file,
        token = token,
        repo_name = "prediction_accidents",
        commit_message = "Airflow : Ajout du dernier encodeur entrainé",
        branch = "main",
    )
    print("Déploiement du model de machine learning en production terminée.")

    xgboost_files = glob.glob('./mlruns/**/xgboost.pkl', recursive=True)
    most_recent_xgboost_file = max(xgboost_files, key=os.path.getmtime)
    upload_to_github(
        timestamp = timestamp,
        input_path = most_recent_xgboost_file,
        token = token,
        repo_name = "prediction_accidents",
        commit_message = "Airflow : Ajout du dernier model entrainé",
        branch = "main",
    )

In [ ]:
data_deployment_func()

Initialisation du déploiement du model de machine learning en production...
Préparation pour l'envoi à Github...
Le fichier 'src/app/api/20230515_230025_targetencoder.pkl' a été téléchargé avec succès sur le dépôt Github 'prediction_accidents'
Déploiement du model de machine learning en production terminée.
Préparation pour l'envoi à Github...
Le fichier 'src/app/api/20230515_230025_xgboost.pkl' a été téléchargé avec succès sur le dépôt Github 'prediction_accidents'


# **Data monitoring**

In [43]:
#data_monitoring.py
def make_tarfile(file_extensions, timestamp):
    target_dir = os.getcwd()

    for file_extension in file_extensions:
        files_to_archive = glob.glob(os.path.join(target_dir, '*' + file_extension))

        for filename in files_to_archive:
            print(f"Commencer la création de l'archive pour {filename}...")
            
            archive_name = os.path.join(target_dir, timestamp + os.path.basename(filename) + '.tar.gz')
            print(f"Le nom de l'archive sera {archive_name}")
            
            with tarfile.open(archive_name, "w:gz") as tar:
                print(f"Ajout de {filename} à l'archive...")
                tar.add(filename, arcname=os.path.basename(filename))
            print(f"Archive {archive_name} créée avec succès.")


def upload_to_github(file_types, token, repo_name, commit_message, branch, file_path_base):
    if isinstance(file_types, str):
        file_types = [file_types]
    print("Préparation pour l'envoi à Github...")
    g = Github(token)
    repo = g.get_user().get_repo(repo_name)

    input_path = os.getcwd()

    for file_type in file_types:
        if os.path.isdir(file_type):
            dir_to_walk = os.path.join(input_path, file_type)
        else:
            dir_to_walk = input_path

        for dirpath, dirnames, filenames in os.walk(dir_to_walk):
            print(f"Dossier actuel : '{dirpath}', sous-dossiers : {dirnames}")
            if not os.path.isdir(file_type):
                filenames = fnmatch.filter(filenames, file_type)
            for filename in filenames:
                local_file_path = os.path.join(dirpath, filename)
                rel_path = os.path.relpath(local_file_path, input_path).replace('\\', '/')
                
                if rel_path.startswith("mlruns/0"):
                    continue

                if file_type == "mlruns" and rel_path.startswith("mlruns/"):
                    rel_path = rel_path[len("mlruns/"):]
                github_file_path = f"{file_path_base}/{rel_path}"
                with open(local_file_path, "rb") as file:
                    content = file.read()
                try:
                    repo.create_file(
                        path=github_file_path,
                        message=commit_message,
                        content=content,
                        branch=branch
                    )
                except Exception as e:
                    print(f"Erreur lors de la création du fichier '{github_file_path}' sur Github : {str(e)}")
    print(f"Les fichiers ont été téléchargés avec succès sur le dépôt Github '{repo_name}'")


def delete_files(file_extensions):
    for file_extension in file_extensions:
        files_to_delete = glob.glob(f"*{file_extension}")
        for filename in files_to_delete:
            try:
                os.remove(filename)
                print(f"Fichier {filename} supprimé avec succès.")
            except Exception as e:
                print(f"Erreur lors de la suppression du fichier {filename} : {str(e)}")

In [44]:
## Tâche pour enregistrer les données de l'expérience
def data_monitoring_func():
    print("Initialisation de l'enregistrement des données de l'expérience...")
    timestamp = time.strftime('%Y%m%d_%H%M%S_')
    file_extensions = ['csv', 'html']
    make_tarfile(file_extensions, timestamp)

    token = os.getenv("GITHUB_TOKEN")

    upload_to_github(
        file_types="*.csv.tar.gz",
        token=token,
        repo_name="prediction_accidents",
        commit_message="Airflow : Ajout des données du dernier entrainement",
        branch="dev",
        file_path_base="data",
    )

    upload_to_github(
        file_types="*.html.tar.gz",
        token=token,
        repo_name="prediction_accidents",
        commit_message="Airflow : Ajout du dernier rapport d'analyse exploratoire",
        branch="dev",
        file_path_base="reports",
    )

    upload_to_github(
        file_types="mlruns",
        token=token,
        repo_name="prediction_accidents",
        commit_message="Airflow : Ajout des artifacts mlflow du dernier entrainement",
        branch="dev",
        file_path_base="models/mlflow/mlruns",
    )

    file_extensions = ['.txt', '.csv', '.html', '.pkl', '.tar.gz']
    delete_files(file_extensions)
    print("Enregistrement des données de l'expérience terminée.")

In [45]:
data_monitoring_func()

Initialisation de l'enregistrement des données de l'expérience...
Préparation pour l'envoi à Github...
Dossier actuel : '/content/drive/MyDrive/Projet MLOps', sous-dossiers : ['mlruns']
Dossier actuel : '/content/drive/MyDrive/Projet MLOps/mlruns', sous-dossiers : ['0', '.trash', '165545817927485843', 'models']
Dossier actuel : '/content/drive/MyDrive/Projet MLOps/mlruns/0', sous-dossiers : []
Dossier actuel : '/content/drive/MyDrive/Projet MLOps/mlruns/.trash', sous-dossiers : []
Dossier actuel : '/content/drive/MyDrive/Projet MLOps/mlruns/165545817927485843', sous-dossiers : ['464377d6d04d4af0ae103750730db043']
Dossier actuel : '/content/drive/MyDrive/Projet MLOps/mlruns/165545817927485843/464377d6d04d4af0ae103750730db043', sous-dossiers : ['metrics', 'params', 'artifacts', 'tags']
Dossier actuel : '/content/drive/MyDrive/Projet MLOps/mlruns/165545817927485843/464377d6d04d4af0ae103750730db043/metrics', sous-dossiers : []
Dossier actuel : '/content/drive/MyDrive/Projet MLOps/mlruns/16

# **Mlflow**

In [ ]:
# Configuration et ouverture d'un tunnel ngrok
ngrok.kill()

NGROK_AUTH_TOKEN = "2P4cOaZTTmH9wl5kjQL3Qnn9NIl_6uFwdWZzo8EF6jdLCLef6"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)
print("Interface utilisateur MLflow Tracking :", ngrok_tunnel.public_url)

Interface utilisateur MLflow Tracking : https://9bc9-35-230-183-144.ngrok-free.app


In [ ]:
# Lancement de l'interface utilisateur de mlflow
!mlflow ui

[2023-05-15 23:24:22 +0000] [14405] [INFO] Starting gunicorn 20.1.0
[2023-05-15 23:24:22 +0000] [14405] [INFO] Listening at: http://127.0.0.1:5000 (14405)
[2023-05-15 23:24:22 +0000] [14405] [INFO] Using worker: sync
[2023-05-15 23:24:22 +0000] [14406] [INFO] Booting worker with pid: 14406
[2023-05-15 23:24:22 +0000] [14407] [INFO] Booting worker with pid: 14407
[2023-05-15 23:24:22 +0000] [14408] [INFO] Booting worker with pid: 14408
[2023-05-15 23:24:22 +0000] [14409] [INFO] Booting worker with pid: 14409

[2023-05-15 23:25:17 +0000] [14405] [INFO] Handling signal: int
Aborted!
[2023-05-15 23:25:17 +0000] [14406] [INFO] Worker exiting (pid: 14406)
[2023-05-15 23:25:17 +0000] [14407] [INFO] Worker exiting (pid: 14407)
[2023-05-15 23:25:17 +0000] [14409] [INFO] Worker exiting (pid: 14409)
[2023-05-15 23:25:17 +0000] [14408] [INFO] Worker exiting (pid: 14408)
[2023-05-15 23:25:18 +0000] [14405] [INFO] Shutting down: Master
